In [ ]:
import pandas as pd
import math as mt
import numpy
import time
import os
import psutil

In [ ]:
details = { 'f': [2,3,3,1,2,2,2,3],
            'g': [3,4,1,3,3,3,1,1],
            'h': [2,2,2,3,3,3,1,2],
            'i': [1,1,2,2,3,3,2,2]}

support = 2

In [ ]:
df = pd.DataFrame(details)
df = df.replace({'?': 0})
df = df.dropna(axis=0) 
print(df.head())


# print(df.columns)
# df = pd.read_csv("obesity.csv", on_bad_lines='skip') # Please Uncomment for external dataset 
# df=df.drop(df.columns[1], axis=1)


   f  g  h  i
0  2  3  2  1
1  3  4  2  1
2  3  1  2  2
3  1  3  3  2
4  2  3  3  3


In [ ]:
class FPTree(object):
    

    def traverse_child_tree(self, i, j, sequence, nodes = None):
        self.head = self.head.get_child_node()
        if self.head.value == sequence[i][j]:
            self.head.set_node_count(self.head.get_node_count() + 1)
        else:
            while self.head.get_right_node() is not None and self.head.get_node_value() != sequence[i][j]:
                self.head = self.head.get_right_node()
            if self.head.get_right_node() is None and self.head.get_node_value() != sequence[i][j]:
                new_right = TreeNode(value=sequence[i][j], count=1, parent=self.head.get_parent_node())
                # self.insert_right(new_right)
                self.head.set_right_node(new_right)
                self.head = new_right
                self.insert_link(sequence[i][j], self.head)
            elif self.head.get_node_value() == sequence[i][j]:
                self.head.set_node_count(self.head.get_node_count() + 1)
 
    def produce(self, sequence, keys= None):
        for i in range(len(sequence)):
            for j in range(len(sequence[i])):
                if self.head.get_child_node() == None:
                    new_child = TreeNode(value=sequence[i][j], count=1, parent=self.head) 
                    self.head.set_child_node(new_child)
                    self.head = new_child
                    self.insert_link(sequence[i][j], self.head)
                else:
                    self.traverse_child_tree(i, j, sequence )
            self.head = self.root
        #Reset Node    
        for item in self.recent.keys():
            while self.recent[item].uplink is not None:
                self.recent[item] = self.recent[item].uplink


    def traverse_sub_child_tree(self, i, j, support, paths, value = False ):
        self.head = self.head.get_child_node()
        if self.head.value == paths[i][j]:
            self.head.count = self.head.count + support[i]
        else:
            while self.head.get_right_node() is not None and self.head.get_node_value() != paths[i][j]:
                self.head = self.head.get_right_node()
            if self.head.get_right_node() is None and self.head.get_node_value() != paths[i][j]:
                new_right = TreeNode(paths[i][j], count=support[i], parent=self.head.get_parent_node())
                # self.insert_right(new_right)
                self.head.set_right_node(new_right)
                self.head = new_right
                self.insert_link(paths[i][j], self.head)
            elif self.head.get_node_value() == paths[i][j]:
                self.head.set_node_count(self.head.get_node_count() + support[i])
 
    def subTreeCreation(self, paths, support, tree = None):
        for i in range(len(paths)):
            for j in range(len(paths[i])):
                if paths[i][j] in self.recent:
                    if self.head.get_child_node() is not None:
                        self.traverse_sub_child_tree(i, j, support, paths)
                    else:
                        new_child = TreeNode(value=paths[i][j], count=support[i], parent=self.head)
                        # self.insert_left(new_child)
                        self.head.set_child_node(new_child)
                        self.head = new_child
                        
                        self.insert_link(paths[i][j], self.head)
            self.head = self.root
        for item in self.recent.keys():
            while self.recent[item].uplink is not None:
                self.recent[item] = self.recent[item].uplink
 
    def single_path_traversal(self, result, a, k = None):
        self.head = self.root
        print("Frequent Patten For Item: ", a)
        self.head = self.head.get_child_node()
        while self.head is not None:
            sup = self.head.count
            s = len(result)
            result.extend(result)
            for i in range(s, len(result)):
                if isinstance(result[i], str):
                    result[i] = [result[i]] + [self.head.get_node_value()]
                else:
                    result[i] = result[i] + [self.head.get_node_value()]
            result.append(self.head.get_node_value())
            self.head = self.head.get_child_node()

        for i in range(len(result)):
            if isinstance(result[i], str):
                result[i] = [result[i]] + [', '.join(a)]
            else:
                result[i] = result[i] + [', '.join(a)]
            print(', '.join(result[i]), ": ", sup)
        print('\n')
         
    def create_base(self, base, paths, supports, item, current, node = False):
        print("Prefix Paths for Item: ", item)
        while current is not None:
            path = []
            start = current 
            current = current.get_parent_node()
            while current.value != []:
                if current.value in base:
                    base[current.value].count = base[current.value].count + start.count
                else:
                    base[current.value] = TreeNode(current.value, start.count)
                path.insert(0, current.value)
                print(current.get_node_value(), end=' ')
                current = current.get_parent_node()
            current = start
            if path != []:
                paths.append(path)
                supports.append(current.count)
                print(current.count)
            # else:
            #     print('\n')
            current = current.link

    

    def insert_link(self, item, node):
        node.uplink = self.recent[item]
        self.recent[item].link = node
        self.recent[item] = node 

    def __init__(self, root=None, data=None, child = None):
        self.root = root
        self.head = root
        self.recent = data  


    @staticmethod
    def perform_FP_growth( self,  keys = [], min_sup = 2, filtered_rows = {}):
        base    = dict()   
        pattern = dict() 
        result  = [] 


        while self.head is not None and self.head.get_right_node() is None:
            self.head = self.head.get_child_node()

        

        if self.head:
            
            for item in self.recent.keys():
                paths = []
                common = []
                supports = []
                if self.recent[item].count < min_sup:
                    continue

                else:
                    current = self.recent[item].link
                    
                    sconditonpbase = dict()
                    print('Starting Mining for ', item)
                    self.create_base(sconditonpbase, paths, supports, item, current)
                    
                    for subkey in list(sconditonpbase.keys()):
                        if sconditonpbase[subkey].count >= min_sup:
                            continue
                        else:
                            del sconditonpbase[subkey]

                    if sconditonpbase == {}:   
                        print("Frequent Patten For Item: ", keys + [item] )
                        print(', '.join(keys + [item]), ": ", self.recent[item].count)

                        print('\n') 
                    else:
                        root = TreeNode(value=[])   
                        btree = FPTree(root, sconditonpbase)
                        if paths == []:
                            continue
                        else:
                            btree.subTreeCreation(paths, supports)
                            base[item] = (paths, supports)
                            keys.append(item)
                            FPTree.perform_FP_growth(btree, keys, min_sup)
                            pattern[item] = (common, min_sup)
                            keys.pop()
                        
        else:
            self.single_path_traversal( result, keys)


In [ ]:
class Preprocesser:

    def sort_on_frequency(self, df_values, sortorder, keys = None):
      sequence = [[] for _ in range(len(df_values))]
      for i in range(len(df_values)):
          for j in sortorder:
            # print(df_values[i], j[0])
            for key in df_values[i]:
              # print(key, j[0])
              if key == j[0]:

                sequence[i].append(key) 
      return sequence

    def encode_data(self, data):  
      for i in range(0, len(data.columns)):
          label = "".join([chr(97 + mt.floor(i / 26)) * (mt.floor(i / 26)), chr(97 + (i % 26))]) 
          for j in range(0, len(data)):
              data.iloc[j, i] = ''.join([label.upper(), str(int(data.iloc[j, i]))]) 
      return data

    def frequency_mapping(self, data, threshold=0):
      mapping = dict()
      for i in range(len(data.columns)):
          unique = list(set(data.iloc[:, i]))
          for item in range(len(unique)):
              if threshold <= len(df.iloc[:, i][data.iloc[:, i] == unique[item]]):
                  mapping[unique[item]] = len(data.iloc[:, i][data.iloc[:, i] == unique[item]]) 
      return mapping
       
    def convert_dict(self, freq_mapping, sorted_freq_mapping): 
      for i in range(len(sortorder)):
        freq_mapping[sortorder[i][0]] = TreeNode(value=sortorder[i][0], count=sortorder[i][1])
      return freq_mapping

    

In [ ]:
class TreeNode(object):
    def __init__(self, value=None, count=0, child_node=None, right_node=None, parent=None, link=None, uplink=None):
        self.child_node  = child_node
        self.right_node = right_node
        self.parent     = parent
        self.value      = value
        self.count      = count
        self.link       = link
        self.uplink     = uplink

    def get_node_value(self):
        value = self.value
        return value

    def get_node_count(self):
        count = self.count
        return count

    def get_child_node(self):
        child_node = self.child_node
        return child_node

    def get_right_node(self):
        right_node = self.right_node
        return right_node

    def set_child_node(self, new_child):
        self.child_node = new_child

    def set_right_node(self, new_right):
        self.right_node = new_right

    def set_parent_node(self, new_parent):
        self.parent = new_parent

    def get_parent_node(self):
        parent = self.parent
        return parent

    def set_node_count(self, new_count):
        self.count = new_count

In [ ]:
pprocess_start_time = time.time()
pprocessor = Preprocesser()


encoded_data = pprocessor.encode_data(df)
data_values  = df.values.tolist()


freq_mapping        = pprocessor.frequency_mapping(encoded_data, support)
sorted_freq_mapping = {k: v for k, v in sorted(freq_mapping.items(), reverse=True, key=lambda item: item[1])}
sortorder           = sorted(sorted_freq_mapping.items(), key=lambda k: (k[1], k[0]), reverse=True)
sorted_encoded_data = pprocessor.sort_on_frequency(data_values, sortorder)


nodeDict = pprocessor.convert_dict(freq_mapping, sortorder) # For creating a link amongst same node values
pprocess_end_time = time.time()
print("Data Preprocessing took %s seconds" % (round(pprocess_end_time - pprocess_start_time, 2)))

Data Preprocessing took 0.02 seconds


In [ ]:
fp_start_time = time.time()
root   = TreeNode(value=[])
fptree = FPTree(root, data = nodeDict)
fptree.produce(sorted_encoded_data) 
FPTree.perform_FP_growth(fptree, min_sup = support)
fp_end_time = time.time()
print("FP Growth Mining took %s seconds" % (round(fp_end_time - fp_start_time, 4)))
print("Total Time for Mining : %s seconds" % (round(fp_end_time - fp_start_time + pprocess_end_time - pprocess_start_time, 2)))


Starting Mining for  A3
Prefix Paths for Item:  A3
C2 1
B1 C2 D2 2
Starting Mining for  C2
Prefix Paths for Item:  C2
D2 2
Frequent Patten For Item:  ['A3', 'C2']
D2, A3, C2 :  2


Starting Mining for  B1
Prefix Paths for Item:  B1
C2 D2 2
Frequent Patten For Item:  ['A3', 'B1']
D2, A3, B1 :  2
D2, C2, A3, B1 :  2
C2, A3, B1 :  2


Starting Mining for  D2
Prefix Paths for Item:  D2
Frequent Patten For Item:  ['A3', 'D2']
A3, D2 :  2


Starting Mining for  A2
Prefix Paths for Item:  A2
B3 C2 1
B3 2
D2 1
Frequent Patten For Item:  ['A2']
B3, A2 :  3


Starting Mining for  B3
Prefix Paths for Item:  B3
C2 1
D2 1
Frequent Patten For Item:  ['B3']
B3 :  4


Starting Mining for  B1
Prefix Paths for Item:  B1
C2 D2 2
A2 D2 1
Frequent Patten For Item:  ['B1']
D2, B1 :  2
D2, C2, B1 :  2
C2, B1 :  2


Starting Mining for  C3
Prefix Paths for Item:  C3
B3 D2 1
A2 B3 2
Frequent Patten For Item:  ['C3']
B3, C3 :  2
B3, A2, C3 :  2
A2, C3 :  2


Starting Mining for  C2
Prefix Paths for Item:  C2
D2

In [ ]:
process = psutil.Process(os.getpid())
print("Total Used Memory for FP Growth Mining: ", end='')
print(process.memory_info().rss)

Total Used Memory for FP Growth Mining: 187871232


In [ ]:
freq_mapping

{'A3': <__main__.TreeNode at 0x7f49e72e44d0>,
 'A2': <__main__.TreeNode at 0x7f49e72e4950>,
 'B3': <__main__.TreeNode at 0x7f49e72e4b90>,
 'B1': <__main__.TreeNode at 0x7f49e72e4150>,
 'C3': <__main__.TreeNode at 0x7f49e72e4c90>,
 'C2': <__main__.TreeNode at 0x7f49e72e4a10>,
 'D3': <__main__.TreeNode at 0x7f49e72e4dd0>,
 'D2': <__main__.TreeNode at 0x7f49e7325d10>,
 'D1': <__main__.TreeNode at 0x7f49e72e4910>}

In [ ]:
encoded_data

,f,g,h,i
0,A2,B3,C2,D1
1,A3,B4,C2,D1
2,A3,B1,C2,D2
3,A1,B3,C3,D2
4,A2,B3,C3,D3
5,A2,B3,C3,D3
6,A2,B1,C1,D2
7,A3,B1,C2,D2


In [ ]:
sorted_encoded_data

[['C2', 'B3', 'A2', 'D1'],
 ['C2', 'A3', 'D1'],
 ['D2', 'C2', 'B1', 'A3'],
 ['D2', 'B3', 'C3'],
 ['B3', 'A2', 'C3', 'D3'],
 ['B3', 'A2', 'C3', 'D3'],
 ['D2', 'A2', 'B1'],
 ['D2', 'C2', 'B1', 'A3']]

In [ ]:
sorted_freq_mapping

{'A2': 4,
 'B3': 4,
 'C2': 4,
 'D2': 4,
 'A3': 3,
 'B1': 3,
 'C3': 3,
 'D3': 2,
 'D1': 2}